In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Retrieve Data

In [2]:
from stockMarket.core import get_tickers_from_index

tickers = get_tickers_from_index('sp500')

In [3]:
from stockMarket.api import populate_contracts

contracts = populate_contracts(tickers)
# contracts = populate_contracts(tickers, update_period="now")
import numpy as np

from stockMarket.core import Screener, LimitScreenerObject, EqualityScreenerObject
from stockMarket.core import growth_total

screener = Screener(contracts)

bank_screener = EqualityScreenerObject('fin_statement_type', equal_to="BANK", description="Bank")
not_bank_screener = EqualityScreenerObject('fin_statement_type', not_equal_to="BANK", description="Bank")

dividend_yield_screener = LimitScreenerObject('dividend_yield', min_value=0.1, description="Dividend Yield")
payout_screener = LimitScreenerObject('payout_ratio', max_value=150, description="Payout Ratio")
price_screener = LimitScreenerObject('price', max_value=40, description="Price")
trailing_pe_screener = LimitScreenerObject('trailing_pe', min_value=0, max_value=30, description="Trailing PE")
forward_pe_screener = LimitScreenerObject('forward_pe', min_value=0, max_value=30, description="Forward PE")
revenue_screener = LimitScreenerObject('revenue', min_value=0, lambda_func= lambda x: growth_total(x)[0], description="Revenue Growth")
operating_cashflow_screener = LimitScreenerObject('operating_cashflow', min_value=0, description="Operating Cashflow")
equity_ratio_screener_not_bank = LimitScreenerObject('equity_ratio', min_value=30, description="Equity Ratio", screener_object=not_bank_screener)
equity_ratio_screener_bank = LimitScreenerObject('equity_ratio', min_value=8, description="Equity Ratio", screener_object=bank_screener)
ebitda_margin_screener = LimitScreenerObject('ebitda_margin', min_value=15, description="EBITDA Margin")
net_income_screener = LimitScreenerObject('net_income', min_value=0, lambda_func=np.mean, description="Min Net Income")

screener_list = [
    # dividend_yield_screener,
    # payout_screener,
    # price_screener,
    # trailing_pe_screener,
    # forward_pe_screener,
    # revenue_screener,
    # operating_cashflow_screener,
    # equity_ratio_screener_not_bank,
    # equity_ratio_screener_bank,
    # ebitda_margin_screener,
    # net_income_screener
]

screened_contracts = screener.screen(screener_list)

In [30]:
from stockMarket.core.contract import Contract, get_data_from
from stockMarket.api import populate_contracts

contract = Contract(ticker='BLK')
contracts = populate_contracts([contract])
print(contracts[0].equity_ratio)

[        nan 32.08759819 24.69275719 19.93592569 19.89479427 20.28789332]


### Perform Screening

In [31]:
import numpy as np

from decorator import decorator

from stockMarket.core import Ranking, RankingObject

@decorator
def min_max(func, min=0, max=0, *args, **kwargs):
    return func(*args, **kwargs), min, max

def rank_equity_ratio(contract):
    equity_ratio = contract.equity_ratio[0]
    if equity_ratio > 30:
        return (equity_ratio, 1)
    if equity_ratio < 10:
        return (equity_ratio, -1)
    else:
        return (equity_ratio, 0)

def netto_margin(contract):
    netto_margin = contract.netto_margin[0]
    if 10 < netto_margin < 20:
        return netto_margin, 1
    if netto_margin > 20:
        return netto_margin, 2
    else:
        return netto_margin, 0

def return_on_assets(contract):
    return_on_assets = contract.return_on_assets[0]
    if 5 < return_on_assets < 10:
        return return_on_assets, 1
    if return_on_assets > 10:
        return return_on_assets, 2
    else:
        return return_on_assets, 0

def goodwill(contract):
    goodwill = contract.goodwill_ratio[0]
    
    if goodwill > 30 or goodwill < 0:
        return goodwill, -1
    else:    
        return goodwill, 0

def gearing(contract):
    gearing = contract.gearing[0]
    if gearing > 60:
        return gearing, -1
    if 20 < gearing < 60:
        return gearing, 0
    else:
        return gearing, 1
    
def peg_trailing_3y(contract):
    trailing_pe = contract.trailing_pe
    
    if len(contract.earnings_per_share) < 4:
        return np.nan, 0
    
    growth = contract.earnings_per_share[0] / contract.earnings_per_share[3]
    peg = contract.peg_trailing_3y
    
    if trailing_pe < 0:
        return peg, -1
    elif growth < 1:
        return peg, -1
    elif peg < 0.8:
        return peg, 1
    elif 0.8 <= peg <= 1.2:
        return peg, 0
    elif np.isnan(peg):
        return peg, 0
    else:
        return peg, -1
    
def pug_trailing_3y(contract):
    price_to_revenue = contract.price_to_revenue[0]
    
    if len(contract.revenue_per_share) < 4:
        return np.nan, 0
    
    growth = contract.revenue_per_share[0] / contract.revenue_per_share[3]
    prg = contract.prg_trailing_3y
    
    if price_to_revenue < 0:
        return prg, -1
    elif growth < 1:
        return prg, -1
    elif prg < 0.8:
        return prg, 1
    elif 0.8 <= prg <= 1.2:
        return prg, 0
    elif np.isnan(prg):
        return prg, 0
    else:
        return prg, -1
    
def pfcg_trailing_3y(contract):
    price_to_free_cashflow = contract.price_to_free_cashflow[0]
    
    if len(contract.free_cashflow_per_share) < 4:
        return np.nan, 0
    
    growth = contract.operating_cashflow[0] / contract.operating_cashflow[3]
    pfcg = contract.pfcg_trailing_3y
    
    if price_to_free_cashflow < 0:
        return pfcg, -1
    elif growth < 1:
        return pfcg, -1
    elif pfcg < 0.8:
        return pfcg, 1
    elif 0.8 <= pfcg <= 1.2:
        return pfcg, 0
    elif np.isnan(pfcg):
        return pfcg, 0
    else:
        return pfcg, -1
    

equity_ratio_ranking = RankingObject(rank_equity_ratio, "Equity Ratio")
netto_margin_ranking = RankingObject(netto_margin, "Netto Margin")
return_on_assets_ranking = RankingObject(return_on_assets, "Return on Assets")
goodwill_ranking = RankingObject(goodwill, "Goodwill")
gearing_ranking = RankingObject(gearing, "Gearing")
peg_trailing_3y_ranking = RankingObject(peg_trailing_3y, "PEG Trailing 3Y")
prg_trailing_3y_ranking = RankingObject(pug_trailing_3y, "PRG Trailing 3Y")
pfcg_trailing_3y_ranking = RankingObject(pfcg_trailing_3y, "PFCG Trailing 3Y")
ranking_list = [
    equity_ratio_ranking,
    netto_margin_ranking,
    return_on_assets_ranking,
    goodwill_ranking,
    gearing_ranking,
    # peg_trailing_3y_ranking,
    # prg_trailing_3y_ranking,
    # pfcg_trailing_3y_ranking,
]

maximum_score = 9
minimum_score = -6

ranking = Ranking(screened_contracts, ranking_list, years_back=0)
ranking.rank()
ranking.ranking.to_csv("ranking_contracts.csv", sep='\t', encoding='utf-8', float_format='%.2f')
ranking.ranking

AttributeError: 'str' object has no attribute 'rank'

In [39]:
import numpy as np
import operator

from stockMarket.core import Ranking, RangeRankingObject
from stockMarket.core.ranking import ValueRankingConstraint

equity_ratio_ranking = RangeRankingObject("Equity Ratio", lambda x: x.equity_ratio, [10, 30], [0,1,2])
netto_margin_ranking = RangeRankingObject("Netto Margin", lambda x: x.netto_margin, [10, 20], [0,1,2])
return_on_assets_ranking = RangeRankingObject("Return on Assets", lambda x: x.return_on_assets, [5, 10], [0,1,2])
goodwill_ranking = RangeRankingObject("Goodwill", lambda x: x.goodwill_ratio, [0, 30], [0,1,0])
gearing_ranking = RangeRankingObject("Gearing", lambda x: x.gearing, [20, 60], [2,1,0])

trailing_pe_constraint = ValueRankingConstraint(lambda x: x.price_to_earnings, operator.gt, 0)
peg_trailing_3y_ranking = RangeRankingObject(
    "PEG Trailing 3Y",
    lambda x: x.peg_trailing_3y,
    [0.8, 1.2], [2,1,0],
    constraints=[trailing_pe_constraint]
)

trailing_pr_constraint = ValueRankingConstraint(lambda x: x.price_to_revenue, operator.gt, 0)
prg_trailing_3y_ranking = RangeRankingObject(
    "PRG Trailing 3Y",
    lambda x: x.prg_trailing_3y,
    [0.8, 1.2], [2,1,0],
    constraints=[trailing_pr_constraint]
)

trailing_pfc_constraint = ValueRankingConstraint(lambda x: x.price_to_free_cashflow, operator.gt, 0)
pfcg_trailing_3y_ranking = RangeRankingObject(
    "PFCG Trailing 3Y",
    lambda x: x.pfcg_trailing_3y,
    [0.8, 1.2], [2,1,0],
    constraints=[trailing_pfc_constraint]
)



ranking_list = [
    equity_ratio_ranking,
    netto_margin_ranking,
    return_on_assets_ranking,
    goodwill_ranking,
    gearing_ranking,
    peg_trailing_3y_ranking,
    prg_trailing_3y_ranking,
    pfcg_trailing_3y_ranking,
]



ranking = Ranking(screened_contracts, ranking_list, years_back=0)
ranking.rank()
ranking.ranking.to_csv("ranking_contracts.csv", sep='\t', encoding='utf-8', float_format='%.2f')
ranking.ranking

WRK
WRK
EXR
EXR
BRO
BRO
REGN
REGN
APTV
APTV
GOOGL
GOOGL
BXP
BXP
ADSK
ADSK
MO
MO
CTSH
CTSH
PCAR
PCAR
MCHP
MCHP
JKHY
JKHY
ZION
ZION
BG
BG
VLTO
NFLX
NFLX
HSY
HSY
TER
TER
TFC
TFC
SBUX
SBUX
BF.B
BF.B
EIX
EIX
ATO
ATO
BX
BX
ETN
ETN
BLDR
BLDR
MSFT
MSFT
HSIC
HSIC
LHX
LHX
BIIB
BIIB
IPG
IPG
PTC
PTC
V
V
TXT
TXT
PWR
PWR
MKC
MKC
VMC
VMC
IT
IT
CHRW
CHRW
MSCI
MSCI
VICI
VICI
COST
COST
TDG
TDG
PNW
PNW
PRU
PRU
LNT
LNT
COR
COR
RTX
RTX
LRCX
LRCX
OTIS
OTIS
ROL
ROL
AFL
AFL
SWKS
SWKS
YUM
YUM
CBOE
CBOE
FDX
FDX
AMT
AMT
TJX
TJX
EBAY
EBAY
LKQ
LKQ
EXPE
EXPE
AOS
AOS
KMB
KMB
PPL
PPL
CAT
CAT
MAA
MAA
CTVA
CTVA
SYF
SYF
CAH
CAH
TT
TT
CNC
CNC
WBA
WBA
HAS
HAS
MDLZ
MDLZ
COO
COO
MCO
MCO
GNRC
GNRC
AMCR
AMCR
ADM
ADM
ANSS
ANSS
BBY
BBY
EW
EW
DVA
DVA
MOS
MOS
STE
STE
WAT
WAT
WELL
WELL
CSX
CSX
VZ
VZ
RVTY
RVTY
WHR
WHR
FMC
FMC
SNA
SNA
VRTX
VRTX
WY
WY
OMC
OMC
TGT
TGT
APH
APH
LH
LH
AIG
AIG
RJF
RJF
ORCL
ORCL
XEL
XEL
HCA
HCA
HD
HD
ADP
ADP
TRGP
TRGP
C
C
BAX
BAX
DLTR
DLTR
MKTX
MKTX
LIN
LIN
ZBRA
ZBRA
SYK
SYK
DD
DD
TECH
TECH
HBAN
HBAN
PLD
P

,Name,Sector,Relative Score,Absolute Score,No Data/Constraints/Tot.,Equity Ratio,Equity Ratio Score,Netto Margin,Netto Margin Score,Return on Assets,Return on Assets Score,Goodwill,Goodwill Score,Gearing,Gearing Score
MRNA,"Moderna, Inc.",Healthcare,100.00%,9/9,0/0/5,57.339171,2,66.060311,2,49.462889,2,0.000000,1,-59.703075,2
REGN,"Regeneron Pharmaceuticals, Inc.",Healthcare,100.00%,9/9,0/0/5,77.577915,2,35.639823,2,14.850160,2,0.000000,1,-13.091246,2
GOOG,Alphabet Inc.,Communication Services,100.00%,9/9,0/0/5,70.125717,2,21.203807,2,16.418809,2,11.306140,1,-21.969283,2
ABNB,"Airbnb, Inc.",Consumer Cyclical,100.00%,9/9,0/0/5,34.667664,2,22.538397,2,11.803217,2,11.690647,1,-111.097122,2
MNST,Monster Beverage Corporation,Consumer Defensive,100.00%,9/9,0/0/5,84.140074,2,24.858103,2,17.649111,2,20.277949,1,-39.413481,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SYY,Sysco Corporation,Consumer Defensive,0.00%,0/9,0/0/5,5.120202,0,0.407377,0,0.952238,0,322.149760,0,872.673447,0
DAL,"Delta Air Lines, Inc.",Industrials,0.00%,0/9,0/0/5,9.105246,0,2.605670,0,1.823263,0,148.176846,0,338.453358,0
PANW,"Palo Alto Networks, Inc.",Technology,0.00%,0/9,0/0/5,1.713782,0,-4.853222,0,-2.178951,0,1308.428571,0,429.809524,0
AES,The AES Corporation,Utilities,0.00%,0/9,0/0/5,8.488305,0,-3.671125,0,-1.240785,0,42.065761,0,682.344532,0


### Write Email

In [40]:
from stockMarket.utils import write_email

emails = ["benjaminlantschner@gmail.com", "97gamjak@gmail.com"]
subject = "Stock Market Ranking"
body = ""
attachment = "ranking_contracts.csv"
write_email(emails, subject, body, attachment)